In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import datetime as dt
import sqlite3 as db

In [2]:
github_userName = 'Tanag3r'
ebird_token = 'j6c7l80ga2ib'
db_name = 'trailheadDirectBirds_sous.db'

In [3]:
##connect to database
def connectDB():
    try:
        cnx = db.connect(db_name)
    except Exception as cnxError:
        raise UserWarning(f'Unable to connect to database due to: {cnxError}')
    return cnx

In [38]:
def weeklyAbundance():
    cnx = connectDB()
    try:
        ObsDataset = pd.read_sql('SELECT comName,locId,obsDt,howMany,subId FROM historicObservations WHERE comName == "Canada Jay";',con=cnx,parse_dates=['obsDt']) #Canada Jay filter is for testing purposes only
        ##add month,year and week of year
        #ObsDataset['obsDt_year'] = pd.DatetimeIndex(ObsDataset['obsDt']).year
        #ObsDataset['obsDt_month'] = pd.DatetimeIndex(ObsDataset['obsDt']).month
        ObsDataset['obsDt_week'] = ObsDataset['obsDt'].dt.isocalendar().week
        #group up data
        ObsDataset = ObsDataset.groupby(['comName','locId','obsDt_week'])['howMany'].median()
        ObsDataset = pd.DataFrame(ObsDataset)
        ObsDataset['relativeAbundance'] = ObsDataset['howMany']/ObsDataset['howMany'].mean()
        ObsDataset.sort_values(by=['comName','locId','obsDt_week'],ascending=True,inplace=True)
    except Exception as calcEx:
        raise UserWarning(calcEx)
    return ObsDataset

In [39]:
trx = weeklyAbundance()
trx

howMany  relativeAbundance
comName    locId     obsDt_week                            
Canada Jay L10129014 13              4.0           1.128205
           L12172834 10              6.0           1.692308
                     11              2.0           0.564103
                     40              7.0           1.974359
                     49              2.0           0.564103
                     52              4.0           1.128205
                     53              8.0           2.256410
           L424117   52              3.0           0.846154
           L8312096  1               1.0           0.282051
                     49              1.0           0.282051
                     52              1.0           0.282051